### What this prototype does

* Classical ML (6 models) → regression predictions + metrics + timings

* Deep Learning (Keras) → regression predictions + metrics + learning curves

* Quantum Regression (VQR) on a small subset (for speed) → continuous predictions comparable to the other models, with graceful fallback if Qiskit ML isn’t available.

## Assumptions
• You already have df in memory with these numeric columns:
["Calories_per_person","Fat_grams_per_person","Protein_grams_per_person","GDP_per_capita","Obesity_rate_percent","Undernourishment_rate_percent","Life_expectancy"]
• You’ve imported the usual stack: pandas, numpy, matplotlib, sklearn, xgboost, tensorflow/keras.
• If Qiskit ML isn’t installed, the Quantum block will skip and tell you exactly why.

### 3. FEATURE ENGINEERING, SPLIT & SCALING  

In [6]:
# ============================================================
# 5) FEATURE ENGINEERING, SPLIT & SCALING  (Regression target: Life_expectancy)
# ============================================================
# Goal: Build one consistent pipeline where ALL models (classical, deep learning, quantum)
#       predict the SAME continuous target (life expectancy). This lets us compare apples-to-apples.

import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# -----------------------------
# Select features (X) and target (y)
# -----------------------------
# You can swap the target to 'Obesity_rate_percent' or 'Undernourishment_rate_percent'
# to repeat the entire comparison for those outcomes.
X = df[[
    "Calories_per_person",
    "Fat_grams_per_person",
    "Protein_grams_per_person",
    "GDP_per_capita",
    "Obesity_rate_percent",
    "Undernourishment_rate_percent"
]].copy()

y = df["Life_expectancy"].copy()

# -----------------------------
# Train/validation/test split
# -----------------------------
# Split into train and test so we can estimate out-of-sample generalization.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -----------------------------
# Scale features
# -----------------------------
# Scaling is crucial for models sensitive to feature magnitude (linear/NN/QML).
# Tree models don't need scaling, but for fairness we'll feed the same scaled arrays.
x_scaler = StandardScaler()
X_train_scaled = x_scaler.fit_transform(X_train)
X_test_scaled  = x_scaler.transform(X_test)

KeyboardInterrupt: 

### 4. CLASSICAL MACHINE LEARNING MODELS

In [ ]:
# ============================================================
# 6) CLASSICAL MACHINE LEARNING MODELS (Regression)
# ============================================================
# We train several regressors, time them, and report MSE/RMSE/MAE/R².
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Optional: XGBoost if installed
try:
    from xgboost import XGBRegressor
    xgb_ok = True
except Exception:
    xgb_ok = False

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(random_state=42),
    "Lasso Regression": Lasso(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=300, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
}
if xgb_ok:
    models["XGBoost"] = XGBRegressor(
        objective="reg:squarederror",
        n_estimators=400,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42
    )

results = []   # to collect metrics/timings
pred_store = {}  # to keep predictions per model for quick comparison

for name, model in models.items():
    t0 = time.perf_counter()
    model.fit(X_train_scaled, y_train)
    fit_time = time.perf_counter() - t0

    t1 = time.perf_counter()
    preds = model.predict(X_test_scaled)
    pred_time = time.perf_counter() - t1

    mse  = mean_squared_error(y_test, preds)
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_test, preds)
    r2   = r2_score(y_test, preds)
    results.append([name, "Classical", mse, rmse, mae, r2, fit_time, pred_time])
    pred_store[name] = preds

# Pretty print classical metrics
res_df = pd.DataFrame(results, columns=["Model","Family","MSE","RMSE","MAE","R2","Fit_s","Pred_s"])
print("\n=== Classical ML Results ===")
print(res_df.sort_values("R2", ascending=False).to_string(index=False))

# Show a few prediction vs actual pairs for the current best classical model
best_clf = res_df.sort_values("R2", ascending=False).iloc[0]["Model"]
print(f"\nSample predictions (Classical - {best_clf}) vs actual (first 10 rows):")
for i in range(10):
    print(f"Pred: {pred_store[best_clf][i]:6.2f} | Actual: {y_test.iloc[i]:6.2f}")

### 5. Deep Learning

In [ ]:
# ============================================================
# 7) DEEP LEARNING REGRESSION (Keras)
# ============================================================
# A compact fully connected network with dropout + early stopping.
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

tf.random.set_seed(42)

dl_model = Sequential([
    # input_shape = number of features in X
    Dense(128, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.25),
    Dense(64, activation="relu"),
    Dropout(0.25),
    Dense(1)  # 1 continuous output = regression
])

dl_model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Early stopping to prevent overfitting; restore the best weights on val loss
early_stop = EarlyStopping(
    monitor="val_loss", patience=20, restore_best_weights=True, verbose=0
)

t0 = time.perf_counter()
history = dl_model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,  # 20% of TRAIN used as validation (not touching test!)
    epochs=300,
    batch_size=32,
    callbacks=[early_stop],
    verbose=0
)
dl_fit_time = time.perf_counter() - t0

t1 = time.perf_counter()
dl_preds = dl_model.predict(X_test_scaled, verbose=0).reshape(-1)
dl_pred_time = time.perf_counter() - t1

dl_mse  = mean_squared_error(y_test, dl_preds)
dl_rmse = np.sqrt(dl_mse)
dl_mae  = mean_absolute_error(y_test, dl_preds)
dl_r2   = r2_score(y_test, dl_preds)

res_df = pd.concat([
    res_df,
    pd.DataFrame([["Neural Net (Keras)","Deep Learning", dl_mse, dl_rmse, dl_mae, dl_r2, dl_fit_time, dl_pred_time]],
                 columns=res_df.columns)
], ignore_index=True)

print("\n=== Deep Learning Result ===")
print(res_df[res_df["Model"]=="Neural Net (Keras)"].to_string(index=False))

# Show some NN predictions vs actual
print("\nSample predictions (Deep Learning) vs actual (first 10 rows):")
for i in range(10):
    print(f"Pred: {dl_preds[i]:6.2f} | Actual: {y_test.iloc[i]:6.2f}")

# Plot training curves so you can visually inspect over/underfitting
import matplotlib.pyplot as plt
plt.figure(figsize=(7,5))
plt.plot(history.history["loss"], label="Train MSE")
plt.plot(history.history["val_loss"], label="Val MSE")
plt.title("Neural Network Training Curve (loss=MSE)")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.legend()
plt.show()


ModuleNotFoundError: No module named 'tensorflow.python'

### 6. Quantum ML

In [ ]:
# ============================================================
# 8) QUANTUM MACHINE LEARNING (QML) — Variational Quantum REGRESSOR (VQR)
# ============================================================
# IMPORTANT:
# • Quantum training is expensive; we use a SMALL subset for a meaningful demo.
# • We STANDARDIZE the target for the quantum model to help the optimizer,
#   then inverse-transform predictions back to the original scale.
# • If qiskit-machine-learning (with VQR) is not installed, we skip gracefully.

qml_available = True
try:
    # Backends & utilities
    from qiskit import Aer
    from qiskit.circuit.library import TwoLocal, ZZFeatureMap
    from qiskit.utils import QuantumInstance
    # VQR used to be in different namespaces depending on version
    try:
        from qiskit_machine_learning.algorithms.regressors import VQR
    except Exception:
        from qiskit_machine_learning.algorithms import VQR
    # Optimizers
    from qiskit.algorithms.optimizers import COBYLA
except Exception as e:
    qml_available = False
    qml_import_error = str(e)

if qml_available:
    # -----------------------------
    # Subsample for speed (tune N down if slow; up if you want more capacity)
    # -----------------------------
    Ntrain_q = min(128, X_train_scaled.shape[0])
    Ntest_q  = min(64,  X_test_scaled.shape[0])

    X_q_train = X_train_scaled[:Ntrain_q]
    X_q_test  = X_test_scaled[:Ntest_q]

    # Scale target specifically for quantum regressor (helps the optimizer landscape).
    y_q_scaler = StandardScaler()
    y_q_train  = y_q_scaler.fit_transform(y_train.values[:Ntrain_q].reshape(-1,1)).ravel()

    # -----------------------------
    # Build the quantum model
    # -----------------------------
    # num_qubits must match number of features. We have 6 features → 6 qubits.
    num_qubits = X_q_train.shape[1]

    # Feature map: encodes classical features into a quantum state
    # ZZFeatureMap introduces entanglement and non-linearity; reps controls depth.
    feature_map = ZZFeatureMap(num_qubits=num_qubits, reps=1)

    # Ansatz (variational circuit): learnable parameters adjusted by optimizer
    ansatz = TwoLocal(
        num_qubits=num_qubits,
        rotation_blocks="ry",
        entanglement_blocks="cz",
        entanglement="linear",
        reps=2
    )

    # Quantum simulator backend; statevector is usually faster for small circuits
    backend = Aer.get_backend("aer_simulator_statevector")
    qi = QuantumInstance(backend=backend, shots=None)  # shots=None uses exact statevector exp.

    # Classical optimizer for the variational parameters
    optimizer = COBYLA(maxiter=200, tol=1e-3)

    # Variational Quantum Regressor
    vqr = VQR(
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer,
        quantum_instance=qi
    )

    # -----------------------------
    # Train & predict with timing
    # -----------------------------
    t0 = time.perf_counter()
    vqr.fit(X_q_train, y_q_train)
    q_fit_time = time.perf_counter() - t0

    t1 = time.perf_counter()
    q_pred_scaled = vqr.predict(X_q_test).reshape(-1)
    q_pred_time = time.perf_counter() - t1

    # Inverse-transform back to original life-expectancy scale
    q_preds = y_q_scaler.inverse_transform(q_pred_scaled.reshape(-1,1)).ravel()

    # For proper comparable metrics, take the matching y_test slice
    y_test_qslice = y_test.iloc[:Ntest_q].values

    q_mse  = mean_squared_error(y_test_qslice, q_preds)
    q_rmse = np.sqrt(q_mse)
    q_mae  = mean_absolute_error(y_test_qslice, q_preds)
    q_r2   = r2_score(y_test_qslice, q_preds)

    res_df = pd.concat([
        res_df,
        pd.DataFrame([["VQR (Quantum)","Quantum", q_mse, q_rmse, q_mae, q_r2, q_fit_time, q_pred_time]],
                     columns=res_df.columns)
    ], ignore_index=True)

    print("\n=== Quantum Regression (VQR) Result ===")
    print(res_df[res_df["Model"]=="VQR (Quantum)"].to_string(index=False))

    # Show some quantum predictions vs actual
    print("\nSample predictions (Quantum VQR) vs actual (first 10 rows of its test slice):")
    for i in range(min(10, len(q_preds))):
        print(f"Pred: {q_preds[i]:6.2f} | Actual: {y_test_qslice[i]:6.2f}")

else:
    print("\n[Quantum block skipped] qiskit-machine-learning (VQR) not available:", qml_import_error)

### 7. UNIFIED MODEL COMPARISON TABLE + QUICK RANKINGS

In [ ]:
# ============================================================
# 9) UNIFIED MODEL COMPARISON TABLE + QUICK RANKINGS
# ============================================================
print("\n=== FINAL COMPARISON (higher R² is better; lower RMSE/MAE is better) ===")
display(res_df.sort_values(["R2","RMSE"], ascending=[False, True]).reset_index(drop=True))

best_by_r2 = res_df.sort_values("R2", ascending=False).iloc[0]
print(f"\nBest model by R²: {best_by_r2['Model']} ({best_by_r2['Family']})  "
      f"R²={best_by_r2['R2']:.3f}, RMSE={best_by_r2['RMSE']:.2f}, MAE={best_by_r2['MAE']:.2f}")

# Optional: Visual check — Predicted vs Actual scatter for the two top models
def scatter_pred_vs_actual(y_true, y_pred, title):
    plt.figure(figsize=(5.5,5))
    plt.scatter(y_true, y_pred, alpha=0.6)
    lims = [min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())]
    plt.plot(lims, lims, '--')
    plt.xlabel("Actual Life Expectancy")
    plt.ylabel("Predicted Life Expectancy")
    plt.title(title)
    plt.show()

# For the overall best classical model and the neural net:
scatter_pred_vs_actual(y_test.values, pred_store[best_clf], f"{best_clf} — Pred vs Actual")
scatter_pred_vs_actual(y_test.values, dl_preds, "Neural Net (Keras) — Pred vs Actual")

# If Quantum ran, plot that too on its (smaller) test slice
if qml_available:
    scatter_pred_vs_actual(y_test_qslice, q_preds, "Quantum VQR — Pred vs Actual (subset)")


=== FINAL COMPARISON (higher R² is better; lower RMSE/MAE is better) ===


NameError: name 'res_df' is not defined